In [1]:
import pandas as pd
import polars as pl
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

In [2]:
df: pl.DataFrame = pl.read_csv(
    "Rodent_Inspection_20250410.csv",
    schema_overrides = {"APPROVED_DATE": pl.Utf8, "INSPECTION_DATE": pl.Utf8},
)

In [3]:
# Reformat the DataFrame columns so that they aren't all capitalized by rather than be upper-case
df.columns = list(map(lambda x: x.title(), df.columns))
df

Inspection_Type,Job_Ticket_Or_Work_Order_Id,Job_Id,Job_Progress,Bbl,Boro_Code,Block,Lot,House_Number,Street_Name,Zip_Code,X_Coord,Y_Coord,Latitude,Longitude,Borough,Inspection_Date,Result,Approved_Date,Location,Community Board,Council District,Census Tract,Bin,Nta
str,i64,str,i64,i64,i64,i64,i64,str,str,i64,i64,i64,f64,f64,str,str,str,str,str,i64,i64,i64,i64,str
"""Initial""",11670593,"""PC6530234""",1,2032890025,2,3289,25,"""326""","""EAST 198 STREET""",10458,1015316,255377,40.867727,-73.887461,"""Bronx""","""08/30/2010 03:23:11 PM""","""Passed""","""09/03/2010 10:43:36 AM""","""(40.867726534028, -73.88746110…",7,15,40502,2016678,"""Bedford Park"""
"""Initial""",11758853,"""PC6101553""",1,1013290121,1,1329,121,"""245""","""EAST 55 STREET""",10022,993525,215573,40.758511,-73.967434,"""Manhattan""","""08/18/2011 12:05:54 PM""","""Passed""","""08/19/2011 12:02:56 PM""","""(40.758511490599, -73.96743383…",6,4,10801,1038588,"""East Midtown-Turtle Bay"""
"""Initial""",12504178,"""PC7270050""",1,null,3,3141,20,null,"""MONTIETH STREET""",11206,1002160,194840,40.701451,-73.935407,"""Brooklyn""","""10/10/2018 12:57:02 PM""","""Passed""","""10/11/2018 08:59:21 AM""","""(40.7014506131434, -73.9354065…",null,null,null,null,null
"""Initial""",12560587,"""PC6481130""",1,1021110015,1,2111,15,"""470""","""WEST 165 STREET""",10032,1001506,244337,40.837644,-73.937771,"""Manhattan""","""02/07/2019 12:48:34 PM""","""Passed""","""02/13/2019 10:28:33 AM""","""(40.83764407994, -73.937771402…",12,10,24301,1062635,"""Washington Heights (South)"""
"""Initial""",12345229,"""PC6794074""",1,2031490090,2,3149,90,"""2110""","""RYER AVENUE""",10457,1011881,250411,40.853455,-73.900632,"""Bronx""","""10/16/2017 01:02:51 PM""","""Rat Activity""","""10/27/2017 02:31:42 PM""","""(40.853455091584, -73.90063242…",5,15,381,2013535,"""Mount Hope"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Initial""",14017042,"""PC8475402""",1,2032120051,2,3212,51,"""2344""","""UNIVERSITY AVENUE""",10468,null,null,40.861466,-73.906076,"""Bronx""","""01/24/2025 12:00:16 PM""","""Passed""","""01/27/2025 02:46:37 PM""","""(40.861466053777, -73.90607581…",7,14,253,2014608,"""University Heights (North)-For…"
"""Initial""",14016732,"""PC8475364""",1,2031980055,2,3198,55,"""2344""","""DAVIDSON AVENUE""",10468,null,null,40.859881,-73.904129,"""Bronx""","""01/23/2025 11:46:40 AM""","""Passed""","""01/27/2025 04:08:34 PM""","""(40.859880736746, -73.90412949…",7,14,239,2014265,"""Fordham Heights"""
"""Compliance""",14016226,"""PC8457329""",2,1019480029,1,1948,29,"""301""","""WEST 121 STREET""",10027,null,null,40.807704,-73.953253,"""Manhattan""","""01/23/2025 08:57:18 AM""","""Passed""","""01/27/2025 02:29:15 PM""","""(40.807703542233, -73.95325299…",10,9,257,1059196,"""Harlem (South)"""


In [4]:
# Modify the data columns to actually make them dates
df = df.with_columns([
    pl.col(col).str.strptime(
        pl.Datetime,
        format = '%m/%d/%Y %I:%M:%S %p',
        strict = False
    ) for col in ('Inspection_Date', 'Approved_Date')]
)

df

Inspection_Type,Job_Ticket_Or_Work_Order_Id,Job_Id,Job_Progress,Bbl,Boro_Code,Block,Lot,House_Number,Street_Name,Zip_Code,X_Coord,Y_Coord,Latitude,Longitude,Borough,Inspection_Date,Result,Approved_Date,Location,Community Board,Council District,Census Tract,Bin,Nta
str,i64,str,i64,i64,i64,i64,i64,str,str,i64,i64,i64,f64,f64,str,datetime[μs],str,datetime[μs],str,i64,i64,i64,i64,str
"""Initial""",11670593,"""PC6530234""",1,2032890025,2,3289,25,"""326""","""EAST 198 STREET""",10458,1015316,255377,40.867727,-73.887461,"""Bronx""",2010-08-30 15:23:11,"""Passed""",2010-09-03 10:43:36,"""(40.867726534028, -73.88746110…",7,15,40502,2016678,"""Bedford Park"""
"""Initial""",11758853,"""PC6101553""",1,1013290121,1,1329,121,"""245""","""EAST 55 STREET""",10022,993525,215573,40.758511,-73.967434,"""Manhattan""",2011-08-18 12:05:54,"""Passed""",2011-08-19 12:02:56,"""(40.758511490599, -73.96743383…",6,4,10801,1038588,"""East Midtown-Turtle Bay"""
"""Initial""",12504178,"""PC7270050""",1,null,3,3141,20,null,"""MONTIETH STREET""",11206,1002160,194840,40.701451,-73.935407,"""Brooklyn""",2018-10-10 12:57:02,"""Passed""",2018-10-11 08:59:21,"""(40.7014506131434, -73.9354065…",null,null,null,null,null
"""Initial""",12560587,"""PC6481130""",1,1021110015,1,2111,15,"""470""","""WEST 165 STREET""",10032,1001506,244337,40.837644,-73.937771,"""Manhattan""",2019-02-07 12:48:34,"""Passed""",2019-02-13 10:28:33,"""(40.83764407994, -73.937771402…",12,10,24301,1062635,"""Washington Heights (South)"""
"""Initial""",12345229,"""PC6794074""",1,2031490090,2,3149,90,"""2110""","""RYER AVENUE""",10457,1011881,250411,40.853455,-73.900632,"""Bronx""",2017-10-16 13:02:51,"""Rat Activity""",2017-10-27 14:31:42,"""(40.853455091584, -73.90063242…",5,15,381,2013535,"""Mount Hope"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Initial""",14017042,"""PC8475402""",1,2032120051,2,3212,51,"""2344""","""UNIVERSITY AVENUE""",10468,null,null,40.861466,-73.906076,"""Bronx""",2025-01-24 12:00:16,"""Passed""",2025-01-27 14:46:37,"""(40.861466053777, -73.90607581…",7,14,253,2014608,"""University Heights (North)-For…"
"""Initial""",14016732,"""PC8475364""",1,2031980055,2,3198,55,"""2344""","""DAVIDSON AVENUE""",10468,null,null,40.859881,-73.904129,"""Bronx""",2025-01-23 11:46:40,"""Passed""",2025-01-27 16:08:34,"""(40.859880736746, -73.90412949…",7,14,239,2014265,"""Fordham Heights"""
"""Compliance""",14016226,"""PC8457329""",2,1019480029,1,1948,29,"""301""","""WEST 121 STREET""",10027,null,null,40.807704,-73.953253,"""Manhattan""",2025-01-23 08:57:18,"""Passed""",2025-01-27 14:29:15,"""(40.807703542233, -73.95325299…",10,9,257,1059196,"""Harlem (South)"""


In [5]:
# Investigate Nulls
df.null_count()

Inspection_Type,Job_Ticket_Or_Work_Order_Id,Job_Id,Job_Progress,Bbl,Boro_Code,Block,Lot,House_Number,Street_Name,Zip_Code,X_Coord,Y_Coord,Latitude,Longitude,Borough,Inspection_Date,Result,Approved_Date,Location,Community Board,Council District,Census Tract,Bin,Nta
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,91317,0,0,0,39045,6984,16422,745403,745403,18853,18853,266,17,37,6912,20336,73235,73235,73235,91317,73235


In [6]:
# Just drop the nulls
df = df.drop_nulls('Inspection_Date')

In [7]:
df_grouped_by_borough: pl.DataFrame = df.group_by('Borough')
df_grouped_by_borough_counts: pl.DataFrame = df_grouped_by_borough.len().drop_nulls()
df_grouped_by_borough_counts

Borough,len
str,u32
"""Bronx""",781208
"""Queens""",232667
"""Manhattan""",879704
"""Brooklyn""",793337
"""Staten Island""",62847


In [8]:
# Only keep dates that come after 2009, as before then the data is very sparse and perhaps not accurate (and also any data that somehow appears in the future)
df = df.filter(
    pl.col('Inspection_Date').is_between(pl.datetime(2010, 1, 1), pl.datetime(2025, 4, 10))
)

In [9]:
NUM_OF_BINS = df_grouped_by_borough_counts.shape[0]

RAINBOW_COLORS = ["#ffadad","#ffd6a5","#fdffb6","#caffbf","#9bf6ff","#a0c4ff","#bdb2ff","#ffc6ff"]

RAINBOW_COLOR_SEQ = [RAINBOW_COLORS[i % len(RAINBOW_COLORS)] for i in range(NUM_OF_BINS)]

In [10]:
sightings_per_borough: px.bar = px.bar(
    data_frame = df_grouped_by_borough_counts,
    x = 'Borough',
    y = 'len',
    template = 'plotly_dark',
    labels = {'len': 'Number of Rodent Sightings (Count)'},
    text_auto = True
)

average_rodents: float = df_grouped_by_borough_counts.get_column('len').mean()

sightings_per_borough.update_layout(
    font_family = "Raleway, sans-serif",
    title = dict(text=f'<b>Rodent Counts Across New York City from 2010 to 2025</b><br><sup>During that time span, there was an average of {average_rodents:,.0f} rodents per borough</sup>'),
    legend = dict (
        x = .5,
        y = -.3,
        orientation = 'h',
        yanchor = 'bottom',
        xanchor = 'center'
    )
)

for bar in sightings_per_borough.data:
    bar.marker.color = RAINBOW_COLOR_SEQ

sightings_per_borough

In [11]:
sightings_per_borough.write_image('./figures/average_rodents_per_borough.png', scale = 6, engine = 'kaleido')

In [12]:
# Create a dataframe that is grouped by each month and then also by each borough
yearly_borough_counts: pl.DataFrame = (df
    .sort('Inspection_Date')
    .group_by_dynamic(
        index_column='Inspection_Date',
        every='1y',
        group_by='Borough'
    )
    .agg(
        pl.len().alias('Inspection_Counts')
    )
)

yearly_borough_counts

Borough,Inspection_Date,Inspection_Counts
str,datetime[μs],u32
"""Bronx""",2010-01-01 00:00:00,49361
"""Bronx""",2011-01-01 00:00:00,39363
"""Bronx""",2012-01-01 00:00:00,43302
"""Bronx""",2013-01-01 00:00:00,51592
"""Bronx""",2014-01-01 00:00:00,48324
…,…,…
null,2021-01-01 00:00:00,65
null,2022-01-01 00:00:00,83
null,2023-01-01 00:00:00,46


In [15]:
# Then we can create a pivot table where each row is a data and each column is the borough values
pivoted_counts: pl.DataFrame = yearly_borough_counts.pivot(
    index = 'Inspection_Date',
    on = 'Borough',
    values = 'Inspection_Counts'
)

# Some values are null so we can fill those with 0
pivoted_counts = pivoted_counts.fill_null(0)

pivoted_counts = pivoted_counts.sort('Inspection_Date')

# Get the Years extracted from the data
YEARS = pivoted_counts.get_column('Inspection_Date').dt.year()

# Add the mean of each month to the DataFrame
pivoted_counts = pivoted_counts.with_columns(
    pl.Series(name = 'Year', values = YEARS),
    pl.Series(name = 'mean', values = list(map(lambda x: np.mean(x[1:]), pivoted_counts.rows())))
)

# Reorder the columns
pivoted_counts = pivoted_counts.select(['Inspection_Date', 'Year', 'Bronx', 'Queens', 'Brooklyn', 'Manhattan', 'Staten Island', 'mean'])

pivoted_counts

Inspection_Date,Year,Bronx,Queens,Brooklyn,Manhattan,Staten Island,mean
datetime[μs],i32,u32,u32,u32,u32,u32,f64
2010-01-01 00:00:00,2010,49361,17823,20816,57632,6903,25422.5
2011-01-01 00:00:00,2011,39363,20550,32969,55197,6176,25709.166667
2012-01-01 00:00:00,2012,43302,17200,52513,52774,4944,28455.5
2013-01-01 00:00:00,2013,51592,14372,26885,34570,3474,21815.5
2014-01-01 00:00:00,2014,48324,15377,28081,34801,6433,22169.333333
…,…,…,…,…,…,…,…
2021-01-01 00:00:00,2021,25349,7040,38877,31357,2614,17550.333333
2022-01-01 00:00:00,2022,63326,14870,85279,72203,2640,39733.5
2023-01-01 00:00:00,2023,66441,16104,71775,81214,2943,39753.833333


In [16]:
# can also do visualization where we have the mean of each year and then trendlines for the count of each Borough
yearly_rodents_by_borough: go.Figure = go.Figure()

# Adding the mean barchart
yearly_rodents_by_borough.add_trace(
    go.Bar(
        x = YEARS,
        y = pivoted_counts.get_column('mean'),
        marker = dict(color = '#FFFFFF'),
        showlegend = False
    )
)

# Adding the line charts for each borough
for i, borough in enumerate(['Bronx', 'Queens', 'Manhattan', 'Brooklyn', 'Staten Island']):
    data = pivoted_counts.get_column(borough)
    color = RAINBOW_COLOR_SEQ[i]
    yearly_rodents_by_borough.add_trace(
        go.Scatter(
            x = YEARS,
            y = data,
            hovertext = borough,
            name = borough,
            line = dict(color=color, width=3)
        )
    )

# Styling
yearly_rodents_by_borough.update_layout(
    template = 'plotly_dark',
    title = dict(text = '<b>Rodent Inspections By Year For Every Borough</b><br><sub><em>Compared to Mean Inspections Across the City</em></sub>'),
    font_family = "Raleway, sans-serif",
)

yearly_rodents_by_borough

In [17]:
yearly_rodents_by_borough.write_image('./figures/yearly_rodents_per_borough.png', scale = 6, engine = 'kaleido')

In [18]:
# Create a dataframe that is grouped by each month and then also by each borough
monthly_rodent_counts: pl.DataFrame = (df
    .sort('Inspection_Date')
    .group_by_dynamic(
        index_column='Inspection_Date',
        every='1mo',
        group_by='Borough'
    )
    .agg(
        pl.len().alias('Inspection_Counts')
    )
)


# Convert Integer Months to Named Months
MONTHS = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

monthly_rodent_counts = monthly_rodent_counts.with_columns(
    pl.col('Inspection_Date')
    .dt.strftime("%B")
    .alias('Month')
)

monthly_rodent_counts

Borough,Inspection_Date,Inspection_Counts,Month
str,datetime[μs],u32,str
"""Bronx""",2010-01-01 00:00:00,1187,"""January"""
"""Bronx""",2010-02-01 00:00:00,1476,"""February"""
"""Bronx""",2010-03-01 00:00:00,1565,"""March"""
"""Bronx""",2010-04-01 00:00:00,2010,"""April"""
"""Bronx""",2010-05-01 00:00:00,5341,"""May"""
…,…,…,…
null,2024-10-01 00:00:00,4,"""October"""
null,2024-11-01 00:00:00,2,"""November"""
null,2024-12-01 00:00:00,10,"""December"""


In [19]:
# Group the monthly_rodent_counts by month now so we can get the mean
monthly_rodent_counts_grouped_borough: pl.DataFrame = (
    monthly_rodent_counts
    .group_by(['Month', 'Borough'])
    .mean()
).sort('Month')

display(monthly_rodent_counts_grouped_borough)

# Do this overall for every month (regardless of boroughs)
monthly_rodent_counts_grouped_total: pl.DataFrame = (
    monthly_rodent_counts
    .group_by(['Month'])
    .mean()
    .drop([pl.col('Borough'), pl.col('Inspection_Date')])                                    
)

display(monthly_rodent_counts_grouped_total)

Month,Borough,Inspection_Date,Inspection_Counts
str,str,datetime[μs],f64
"""April""","""Staten Island""",2017-07-31 14:24:00,396.933333
"""April""","""Queens""",2017-07-31 14:24:00,1524.666667
"""April""",null,2022-09-30 18:00:00,4.5
"""April""","""Manhattan""",2017-09-30 12:00:00,5928.5
"""April""","""Brooklyn""",2017-07-31 14:24:00,4718.0
…,…,…,…
"""September""",null,2022-09-01 04:48:00,5.6
"""September""","""Queens""",2017-08-31 20:48:00,1307.066667
"""September""","""Brooklyn""",2017-08-31 20:48:00,4202.4


Month,Inspection_Counts
str,f64
"""January""",2393.829268
"""April""",3185.679012
"""May""",3098.025641
"""July""",2779.670886
"""September""",2759.2125
…,…
"""June""",2938.265823
"""October""",2697.85
"""February""",2560.870588


In [20]:
# get all of the medians each of the months (jan, feb, march) and then plot that with the same per-borough approach as above
# can also do visualization where we have the mean of each year and then trendlines for the count of each Borough
average_monthly_rodents_by_borough: go.Figure = go.Figure()

# Adding the mean barchart
average_monthly_rodents_by_borough.add_trace(
    go.Bar(
        x = MONTHS,
        y = monthly_rodent_counts_grouped_total.get_column('Inspection_Counts'),
        marker = dict(color = '#FFFFFF'),
        showlegend = False
    )
)

# Adding the line charts for each borough
for i, borough in enumerate(['Bronx', 'Queens', 'Manhattan', 'Brooklyn', 'Staten Island']):
    data = monthly_rodent_counts_grouped_borough.filter(pl.col('Borough') == borough)
    color = RAINBOW_COLOR_SEQ[i]
    average_monthly_rodents_by_borough.add_trace(
        go.Scatter(
            x = MONTHS,
            y = data.get_column('Inspection_Counts'),
            hovertext = borough,
            name = borough,
            line = dict(color=color, width=3)
        )
    )

# Styling
average_monthly_rodents_by_borough.update_layout(
    template = 'plotly_dark',
    title = dict(text = '<b>Rodent Inspections By Average Monthly Inspections For Every Borough</b><br><sub><em>Compared to Average Inspections Across the City</em></sub>'),
    font_family = "Raleway, sans-serif",
)

average_monthly_rodents_by_borough

In [21]:
average_monthly_rodents_by_borough.write_image('./figures/average_monthly_rodents_per_borough.png', scale = 6, engine = 'kaleido')